# GOALS

* Decide on viewcount threshold to eliminate views?
* Get feature columns working
* Build an LDA model

In [1]:
## testing printing output from console
import subprocess
cmd = [ 'echo', '"Welcome to my PySpark analysis of some StackExchange Data"' ]
output = subprocess.Popen( cmd, stdout=subprocess.PIPE ).communicate()[0]
print(output)

b'"Welcome to my PySpark analysis of some StackExchange Data"\n'


In [2]:
import time
import numpy as np
import pandas as pd
from datetime import datetime

# Load PySpark and Data Structures

In [3]:
%run -i '1-load-pyspark.py'

## array of dataset names to loop through in analysis
data_array = [
    "english",
    "rus_stackoverflow",
    "superuser",
    "stackoverflow",
    "math"
]

data_array = [
    "economics",
    "buddhism",
    "fitness",
    "health",
    "interpersonal"
]

datasets = {}   

The Spark UI is available at: http://192.168.0.26:4040/ and the defaultParallelism is 4


# Load Data

In [4]:
%%time
print(datetime.now().time())
#%run -i '2-load-datasets.py'

09:42:05.999376
CPU times: user 115 µs, sys: 30 µs, total: 145 µs
Wall time: 201 µs


In [5]:
def show_spark_df(df, n=5):
    '''
    function to better print spark df entries
    '''
    display(pd.DataFrame(df.head(n), columns=df.columns))

# Clean Data

In [6]:
%%time
print(datetime.now().time())
#%run -i '3-clean-datasets.py'

09:42:06.018372
CPU times: user 152 µs, sys: 64 µs, total: 216 µs
Wall time: 168 µs


# Define Target and Perform EDA

In [7]:
%%time
print(datetime.now().time())
#%run -i '4-define-target-eda.py'

#NB TO DO: Find threshold to delete low views to make sure users that can vote have seen the question

'''
vc_thresh_data = {}

## finding means of viewcounts across fora

for i in data_array:
    vc_thresh_data[i] = datasets[i].select("viewcount").rdd.flatMap(lambda x: x).mean()

vc_thresh_data'''

09:42:06.031113
CPU times: user 239 µs, sys: 72 µs, total: 311 µs
Wall time: 269 µs


# Export Clean Data with Target

In [8]:
%%time
print(datetime.now().time())
#%run -i '5-export-data.py'

09:42:06.040152
CPU times: user 288 µs, sys: 77 µs, total: 365 µs
Wall time: 308 µs


# Load Cleaned Data

In [9]:
%%time

## load clean data
for i in data_array:
    datasets[i] = (
        spark
        .read
        .load(f'clean-data/{i}.parquet')
)

CPU times: user 3.37 ms, sys: 2.19 ms, total: 5.57 ms
Wall time: 3.93 s


# Train/Test Split Manually

In [10]:
%%time
print(datetime.now().time())
%run -i '6-train-test-split.py'

09:42:09.991450
economics: 0.19688614985403827, from 1214 and 6166
buddhism: 0.20017182130584193, from 932 and 4656
fitness: 0.20499851234751562, from 1378 and 6722
health: 0.18846909805634418, from 863 and 4579
interpersonal: 0.2189300411522634, from 532 and 2430
CPU times: user 29 ms, sys: 8.75 ms, total: 37.7 ms
Wall time: 4.34 s


In [11]:
'''
interesting to see how skewed rus_stackoverflow posts are to more posts in recent years
'''

'\ninteresting to see how skewed rus_stackoverflow posts are to more posts in recent years\n'

# Create Results Dictionary

In [12]:
RESULTS = {}
for i in data_array:
    # capitalise keys
    RESULTS[i.title()] = {}

# Silly Mean Model

In [13]:
from pyspark.sql.functions import array, lit, struct

## create mean dictionaries
y_ravi_means = {}

## calculate the mean of each forum, using ONLY training set
for i in data_array:
    y_ravi_means[i] = train[i].select('y_ravi').rdd.flatMap(lambda x: x).mean()

In [14]:
## import rmse evaluator
from pyspark.ml.evaluation import RegressionEvaluator

## create baselines dictionary
base = {}

## modelling
for i in data_array:

    ## initial variable for timing
    t0 = time.time()
    
    ## train silly mean model by assigning training set mean for test predictions
    test[i] = test[i].withColumn('mean_pred', lit(y_ravi_means[i]))

    ## evaluate silly mean model, ONLY on test set
    evaluator = RegressionEvaluator(metricName="rmse", labelCol="y_ravi", predictionCol="mean_pred")
    base[i] = round( evaluator.evaluate(test[i]), 6)

    print(f"The root-mean-square error of \033[94m{i}'s\033[0m\033[92m mean\033[0m model is {base[i]}")

    ## record time taken
    timet = round( time.time() - t0, 2 )
    
    ## store as dictionary inside RESULTS dictionary, initiating dataset name entries first
    RESULTS[i.title()]['0silly_mean.rmse'] = base[i]
    RESULTS[i.title()]['0silly_mean.timet'] = timet

The root-mean-square error of economics's mean model is 0.037792
The root-mean-square error of buddhism's mean model is 0.015749
The root-mean-square error of fitness's mean model is 0.020767
The root-mean-square error of health's mean model is 0.040167
The root-mean-square error of interpersonal's mean model is 0.008395


In [15]:
def show_save_results(results, filename='final-results.csv'):
    '''
    function to print and export modelling results
    '''
    display(pd.DataFrame.from_dict(results).T)
    print(pd.DataFrame.from_dict(results).T.to_latex())
    pd.DataFrame.from_dict(results).T.to_csv(filename)

In [16]:
#show_save_results(RESULTS)

,0silly_mean.rmse,0silly_mean.timet
Economics,0.037792,0.46
Buddhism,0.015749,0.17
Fitness,0.020767,0.19
Health,0.040167,0.17
Interpersonal,0.008395,0.21


\begin{tabular}{lrr}
\toprule
{} &  0silly\_mean.rmse &  0silly\_mean.timet \\
\midrule
Economics     &          0.037792 &               0.46 \\
Buddhism      &          0.015749 &               0.17 \\
Fitness       &          0.020767 &               0.19 \\
Health        &          0.040167 &               0.17 \\
Interpersonal &          0.008395 &               0.21 \\
\bottomrule
\end{tabular}



# Viewcount Model

In [24]:
%%time
print(datetime.now().time())
#3min 56s

from pyspark.ml.pipeline import Pipeline
from pyspark.ml.feature import CountVectorizer, StandardScaler, VectorAssembler, VectorSlicer

########################
##### CHOOSE FEATS ##### can't get date right
########################

## define features to predict on
target = 'y_ravi'
numic_variables = ['viewcount']
datet_variables = ['clean_date']

## numerical columns
numic_assembler = VectorAssembler(inputCols=numic_variables, outputCol='numic_data') # have to put in single col
standardiser = StandardScaler(inputCol='numic_data', outputCol='numic_data_std')    
numic_pipeline = Pipeline(stages=[numic_assembler, standardiser])

## date columns
datet_assembler = VectorAssembler(inputCols=datet_variables, outputCol='datet_data')

## create processing pipeline
process_assembler = VectorAssembler(inputCols=['numic_data'], #inputCols=['datet_data']
                                    outputCol='features') 
process_pipeline = Pipeline(stages=[numic_pipeline, process_assembler])

########################
##### CHOOSE MODEL #####
########################

## linear regression on just viewcount
from pyspark.ml.regression import LinearRegression
    
lr = LinearRegression(#maxIter=100, # this doesn't change anything
                      #regParam=0.3, # using regularisation parameter here useless since there is one feature
                      #elasticNetParam=0.8,
                      featuresCol="features",
                      labelCol=target,
                      predictionCol="viewcount_pred")

## make final pipeline
final_pipeline = Pipeline(stages=[process_pipeline, lr])

## import methods for tuning
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

## set up grid for parameter tuning: 
#.addGrid(lr.regParam, [1e-3, 1.])
#.addGrid(lr.elasticNetParam, [1e-3, 1.])
paramGrid = ParamGridBuilder() \
    .build()

## set up rmse evaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="y_ravi", predictionCol="viewcount_pred")

## set up cross validation for parameter tuning
crossval = CrossValidator(estimator=final_pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=2)

## modelling
for i in data_array:
    
    ## initial variable for timing
    t0 = time.time()
    
    ## this fits on train, then predicts on test, using cross val
    rmse = round( evaluator.evaluate(crossval.fit(train[i]).transform(test[i])), 6)
        
    print(f"The root-mean-square error of \033[94m{i}'s\033[0m\033[92m viewcount\033[0m model is {rmse}")

    ## calculate improvement over median baseline
    impr = round( (rmse/base[i] - 1)*-100, 3 )
    
    ## record time taken
    timt = round( time.time() - t0, 2 )

    ## store as dictionary inside RESULTS dictionary
    RESULTS[i.title()]['1viewcount.rmse'] = rmse
    RESULTS[i.title()]['1viewcount.v.imprv'] = impr
    RESULTS[i.title()]['1viewcount.timet'] = timt
    
## record results
#show_save_results(RESULTS)

09:43:26.248420
The root-mean-square error of economics's viewcount model is 0.037722
The root-mean-square error of buddhism's viewcount model is 0.015799
The root-mean-square error of fitness's viewcount model is 0.020627
The root-mean-square error of health's viewcount model is 0.040102
The root-mean-square error of interpersonal's viewcount model is 0.008177
CPU times: user 954 ms, sys: 262 ms, total: 1.22 s
Wall time: 17.9 s


In [25]:
'''
Interesting that there are different improvements of viewcount over mean-only prediciton
'''

'\nInteresting that there are different improvements of viewcount over mean-only prediciton\n'

# Begin Feature Engineering

In [26]:
## garbage collector to speed up computation
import gc
collected = gc.collect()
print('Garbage collector: collected %d objects.' % collected)

Garbage collector: collected 799 objects.


In [27]:
#######################
###### TOKENISER ######
#######################

import nltk
from nltk.stem import PorterStemmer
ps = PorterStemmer()
from pyspark import keyword_only  ## < 2.0 -> pyspark.ml.util.keyword_only
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

## custom transformer for nltk tokenisation
class NLTKWordPunctTokeniser(Transformer, HasInputCol, HasOutputCol):

    @keyword_only
    def __init__(self, inputCol=None, outputCol=None, stopwords=None):
        super(NLTKWordPunctTokeniser, self).__init__()
        self.stopwords = Param(self, "stopwords", "")
        self._setDefault(stopwords=set())
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCol=None, outputCol=None, stopwords=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)

    def setStopwords(self, value):
        self._paramMap[self.stopwords] = value
        return self

    def getStopwords(self):
        return self.getOrDefault(self.stopwords)

    def _transform(self, dataset):
        stopwords = self.getStopwords()

        def f(s):
            # get tokens with separate punctuation
            tokens = nltk.tokenize.wordpunct_tokenize(s)
            # sort out russian stopwords
            #if ('я' in tokens):
            #    tokens = [t for t in tokens if t not in nltk.corpus.stopwords.words('russian')]
            # remove english stopwords
            tokens = [t for t in tokens if t not in stopwords]
            # remove single letters
            #tokens = [t for t in tokens if not len(t)==1] # this removes single punctuation as well
            # stemming the words
            tokens = [ps.stem(t) for t in tokens]
            # convert to lower case
            return [t.lower() for t in tokens]
    
        t = ArrayType(StringType())
        out_col = self.getOutputCol()
        in_col = dataset[self.getInputCol()]
        return dataset.withColumn(out_col, udf(f, t)(in_col))

In [37]:
%%time
print(datetime.now().time())
# ENGLISH
# 8min 56s for no CV and no GRIDSEARCH
# 17min 10s for 3-CV and no GRIDSEARCH
# SMALL DATASETS
# 36min 55s for no CV and no GRIDSEARCH
# 12min 35s for 3-CV and no GRIDSEARCH
# 8min 32s for 2-CV and no GRIDSEARCH

from pyspark.ml.pipeline import Pipeline
from pyspark.ml.feature import CountVectorizer, StandardScaler, VectorAssembler, VectorSlicer

########################
##### CHOOSE FEATS ##### can't get date right
########################

## define features to predict on
target = 'y_ravi'
textt_variables = ['title', 'clean_body']
datet_variables = ['clean_date']

## date columns
datet_assembler = VectorAssembler(inputCols=datet_variables, outputCol='datet_data')

## textual columns
# tokenising text cols with custom transformer
nltk_tokeniser_body = NLTKWordPunctTokeniser(
    inputCol='clean_body', outputCol='body_words',  
    stopwords=set(nltk.corpus.stopwords.words('english')))

nltk_tokeniser_title = NLTKWordPunctTokeniser(
    inputCol='title', outputCol='titl_words',  
    stopwords=set(nltk.corpus.stopwords.words('english')))

# count occurence of tokens, i.e. create dfm
cnt_vectrizr_body = CountVectorizer(inputCol='body_words', outputCol='body_feats', minDF=2)
cnt_vectrizr_title = CountVectorizer(inputCol='titl_words', outputCol='titl_feats', minDF=2)

## create processing pipeline
process_assembler = VectorAssembler(inputCols=['body_feats', 'titl_feats'], #inputCols=['datet_data']
                                    outputCol='features') 
process_pipeline = Pipeline(stages=[  #inputCols=['datet_data']
    nltk_tokeniser_body, 
    nltk_tokeniser_title,
    #token_counter_body,
    #token_counter_title,
    cnt_vectrizr_body,
    cnt_vectrizr_title,
    process_assembler
])

########################
##### CHOOSE MODEL #####
########################

## linear regression on just viewcount
from pyspark.ml.regression import LinearRegression
    
lr = LinearRegression(maxIter=100,
                      regParam=1,
                      elasticNetParam=0.001,
                      featuresCol='features',
                      labelCol=target,
                      predictionCol='tokens_pred')

## make final pipeline
final_pipeline = Pipeline(stages=[process_pipeline, lr])

## import methods for tuning
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

## set up grid for parameter tuning: 
'''NEEDED, BUT IMMENSELY SLOWING DOWN'''
# Ravi et al use L2, aka ridge, aka elasticNetParam=0
# regParam is the value of lambda
#.addGrid(lr.elasticNetParam, [1e-3, 1.])
#.addGrid(lr.regParam, [1e-3, 1.])
paramGrid = ParamGridBuilder() \
    .build()


## set up rmse evaluator
evaluator = RegressionEvaluator(metricName='rmse', labelCol='y_ravi', predictionCol='tokens_pred')

## set up cross validation for parameter tuning
'''DEFINITELY SLOWING DOWN'''
crossval = CrossValidator(estimator=final_pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=2)
models = {}

## modelling
for i in data_array:
    
    ## initial variable for timing
    t0 = time.time()
    
    ## fit on training set with CV, store model and parameters, predict on test set
    cvmodel = crossval.fit(train[i])
    models[i] = cvmodel
    rmse = round( evaluator.evaluate(cvmodel.transform(test[i])), 6)
    
    print(f"The root-mean-square error of \033[94m{i}'s\033[0m\033[92m tokens\033[0m model is {rmse}")

    ## calculate improvement over median baseline
    impr = round( (rmse/base[i] - 1)*-100, 3 )
    
    ## record time taken
    timt = round( time.time() - t0, 2 )

    ## store as dictionary inside RESULTS dictionary
    RESULTS[i.title()]['2tokens.rmse'] = rmse
    RESULTS[i.title()]['2tokens.v.imprv'] = impr
    RESULTS[i.title()]['2tokens.timet'] = timt
    
## record results
show_save_results(RESULTS, '2cv-0.001elastic-1reg.csv')

09:59:37.488166
The root-mean-square error of economics's tokens model is 0.037769
The root-mean-square error of buddhism's tokens model is 0.015749
The root-mean-square error of fitness's tokens model is 0.020767
The root-mean-square error of health's tokens model is 0.040057
The root-mean-square error of interpersonal's tokens model is 0.008395


,0silly_mean.rmse,0silly_mean.timet,1viewcount.rmse,1viewcount.timet,1viewcount.v.imprv,2tokens.rmse,2tokens.timet,2tokens.v.imprv
Economics,0.037792,0.46,0.037722,5.91,0.185,0.037769,115.50,0.061
Buddhism,0.015749,0.17,0.015799,3.59,-0.317,0.015749,76.51,-0.000
Fitness,0.020767,0.19,0.020627,3.35,0.674,0.020767,114.40,-0.000
Health,0.040167,0.17,0.040102,2.54,0.162,0.040057,67.47,0.274
Interpersonal,0.008395,0.21,0.008177,2.45,2.597,0.008395,87.88,-0.000


\begin{tabular}{lrrrrrrrr}
\toprule
{} &  0silly\_mean.rmse &  0silly\_mean.timet &  1viewcount.rmse &  1viewcount.timet &  1viewcount.v.imprv &  2tokens.rmse &  2tokens.timet &  2tokens.v.imprv \\
\midrule
Economics     &          0.037792 &               0.46 &         0.037722 &              5.91 &               0.185 &      0.037769 &         115.50 &            0.061 \\
Buddhism      &          0.015749 &               0.17 &         0.015799 &              3.59 &              -0.317 &      0.015749 &          76.51 &           -0.000 \\
Fitness       &          0.020767 &               0.19 &         0.020627 &              3.35 &               0.674 &      0.020767 &         114.40 &           -0.000 \\
Health        &          0.040167 &               0.17 &         0.040102 &              2.54 &               0.162 &      0.040057 &          67.47 &            0.274 \\
Interpersonal &          0.008395 &               0.21 &         0.008177 &              2.45 &              

In [53]:
## check predictions aren't constant
models['health'].transform(test['health']).select('tokens_pred').take(10)

[Row(tokens_pred=0.034058891041456055),
 Row(tokens_pred=0.03405929314867255),
 Row(tokens_pred=0.034079450138317174),
 Row(tokens_pred=0.0337468957965947),
 Row(tokens_pred=0.034174996262087545),
 Row(tokens_pred=0.03462413748313258),
 Row(tokens_pred=0.03393305826306672),
 Row(tokens_pred=0.03410357979472746),
 Row(tokens_pred=0.03410431747270664),
 Row(tokens_pred=0.03411632288668403)]

In [ ]:
"""why the heck do fitness and interpersonal have constant predictions - it's not the parameters or the size of the data"""

In [56]:
## have a look at CV models params
list(zip(models['health'].avgMetrics, paramGrid))

[(0.035319195593229886, {})]

In [92]:
## extract best parameters
for i in data_array:
    # elasticnet
    ela_key = list(models[i].bestModel.stages[-1].extractParamMap().keys())[1]
    ela_param = models[i].bestModel.stages[-1].extractParamMap()[ela_key]
    # reg'sation
    reg_key = list(models[i].bestModel.stages[-1].extractParamMap().keys())[9]
    reg_param = models[i].bestModel.stages[-1].extractParamMap()[reg_key]
    print(i)
    print(f'elastic net: {ela_param})
    

In [93]:
list(models['economics'].bestModel.stages[-1].extractParamMap().keys())

[Param(parent='LinearRegression_33b24407825c', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2)'),
 Param(parent='LinearRegression_33b24407825c', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty'),
 Param(parent='LinearRegression_33b24407825c', name='epsilon', doc='The shape parameter to control the amount of robustness. Must be > 1.0.'),
 Param(parent='LinearRegression_33b24407825c', name='featuresCol', doc='features column name'),
 Param(parent='LinearRegression_33b24407825c', name='fitIntercept', doc='whether to fit an intercept term'),
 Param(parent='LinearRegression_33b24407825c', name='labelCol', doc='label column name'),
 Param(parent='LinearRegression_33b24407825c', name='loss', doc='The loss function to be optimized. Supported options: squaredError, huber. (Default squaredError)'),
 Param(parent='LinearRegression_33b24407825c', name='maxIter', doc

In [ ]:
models['economics'].bestModel.stages[1].extractParamMap().items()[0]

In [63]:
## look at the best parameters
for i in data_array:
    print(f'The best model parameters for \033[94m{i}\033[0m are \
    {models[i].bestModel.stages[1].extractParamMap()}')

The best model parameters for economics are     {Param(parent='LinearRegression_33b24407825c', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2)'): 2, Param(parent='LinearRegression_33b24407825c', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty'): 0.001, Param(parent='LinearRegression_33b24407825c', name='epsilon', doc='The shape parameter to control the amount of robustness. Must be > 1.0.'): 1.35, Param(parent='LinearRegression_33b24407825c', name='featuresCol', doc='features column name'): 'features', Param(parent='LinearRegression_33b24407825c', name='fitIntercept', doc='whether to fit an intercept term'): True, Param(parent='LinearRegression_33b24407825c', name='labelCol', doc='label column name'): 'y_ravi', Param(parent='LinearRegression_33b24407825c', name='loss', doc='The loss function to be optimized. Supported options: squaredError, huber. (Defau

In [71]:
for i in data_array:
    for key, value in models[i].bestModel.stages[1].extractParamMap().items():
        print (key)

LinearRegression_33b24407825c__aggregationDepth
LinearRegression_33b24407825c__elasticNetParam
LinearRegression_33b24407825c__epsilon
LinearRegression_33b24407825c__featuresCol
LinearRegression_33b24407825c__fitIntercept
LinearRegression_33b24407825c__labelCol
LinearRegression_33b24407825c__loss
LinearRegression_33b24407825c__maxIter
LinearRegression_33b24407825c__predictionCol
LinearRegression_33b24407825c__regParam
LinearRegression_33b24407825c__solver
LinearRegression_33b24407825c__standardization
LinearRegression_33b24407825c__tol
LinearRegression_33b24407825c__aggregationDepth
LinearRegression_33b24407825c__elasticNetParam
LinearRegression_33b24407825c__epsilon
LinearRegression_33b24407825c__featuresCol
LinearRegression_33b24407825c__fitIntercept
LinearRegression_33b24407825c__labelCol
LinearRegression_33b24407825c__loss
LinearRegression_33b24407825c__maxIter
LinearRegression_33b24407825c__predictionCol
LinearRegression_33b24407825c__regParam
LinearRegression_33b24407825c__solver


In [ ]:
param_dict['economics'].bestModel.stages[1].extractParamMap()

In [ ]:
df_ = spark.createDataFrame(datasets['economics'].take(5))

In [ ]:
show_spark_df(df_)

In [ ]:
word_feat_list = tester4.transform( tester3.fit(tester.transform(df_)).transform(tester.transform(df_)) )

In [ ]:
word_feat_list

In [ ]:
word_feat_list.select('features').collect()

In [ ]:
show_spark_df(word_feat_list)

In [ ]:
from pyspark.mllib.linalg import Vectors
def as_mllib_vector(v):
    return Vectors.sparse(v.size, v.indices, v.values)

In [ ]:
word_feat_list.select("features").rdd.map(lambda r: r[0][0]).collect()

In [ ]:
word_feat_list.select("features").rdd.map(lambda r: r[0]).map(lambda r: (r,1)).reduceByKey(lambda a,b: a+b).collect()

In [ ]:
tester = NLTKWordPunctTokeniser(
    inputCol='title', outputCol='title_words',  
    stopwords=set(nltk.corpus.stopwords.words('english')))

tester2 = NLTKWordPunctTokeniser(
    inputCol='clean_body', outputCol='body_words',  
    stopwords=set(nltk.corpus.stopwords.words('english')))

In [ ]:
tester3 = CountVectorizer(inputCol="title_words", outputCol="features")

In [ ]:
tester4 = NLTKCounter(inputCol='features', outputCol='final_features')

In [ ]:
########################
###### SENTENISER ######
########################

import nltk
from nltk.stem import PorterStemmer
ps = PorterStemmer()
from pyspark import keyword_only  ## < 2.0 -> pyspark.ml.util.keyword_only
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

## custom transformer for nltk tokenisation
class NLTKSenteniser(Transformer, HasInputCol, HasOutputCol):

    @keyword_only
    def __init__(self, inputCol=None, outputCol=None, stopwords=None):
        super(NLTKWordPunctTokenizer, self).__init__()
        self.stopwords = Param(self, "stopwords", "")
        self._setDefault(stopwords=set())
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCol=None, outputCol=None, stopwords=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)

    def setStopwords(self, value):
        self._paramMap[self.stopwords] = value
        return self

    def getStopwords(self):
        return self.getOrDefault(self.stopwords)

    def _transform(self, dataset):
        stopwords = self.getStopwords()

        def f(s):
            # get tokens with separate punctuation
            tokens = nltk.tokenize.wordpunct_tokenize(s)
            # sort out russian stopwords
            if ('я' in tokens):
                tokens = [t for t in tokens if t not in nltk.corpus.stopwords.words('russian')]
            # remove english stopwords
            tokens = [t for t in tokens if t not in stopwords]
            # remove single letters
            tokens = [t for t in tokens if not len(t)==1]
            # stemming the words
            tokens = [ps.stem(t) for t in tokens]
            # convert to lower case
            return [t.lower() for t in tokens]
    
        t = ArrayType(StringType())
        out_col = self.getOutputCol()
        in_col = dataset[self.getInputCol()]
        return dataset.withColumn(out_col, udf(f, t)(in_col))

In [ ]:
#######################
####### COUNTER #######
#######################

import nltk
from pyspark import keyword_only  ## < 2.0 -> pyspark.ml.util.keyword_only
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

## custom transformer for nltk tokenisation
class NLTKCounter(Transformer, HasInputCol, HasOutputCol):
    '''
    Takes in cnt_vectrizr to count number of tokens per question
    '''

    @keyword_only
    def __init__(self, inputCol=None, outputCol=None):
        super(NLTKCounter, self).__init__()
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

    @keyword_only
    def setParams(self, inputCol=None, outputCol=None):
        kwargs = self._input_kwargs
        return self._set(**kwargs)

    def _transform(self, dataset):

        def f(s):
            return 1
    
        t = ArrayType(StringType())
        out_col = self.getOutputCol()
        in_col = dataset[self.getInputCol()]
        return dataset.withColumn(out_col, udf(f, t)(in_col))

In [ ]:
a

In [ ]:
## import elements from natural language toolkit
import nltk
#nltk.download('all') # uncomment after first run as admin check
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop_words = set(stopwords.words('english'))
table = str.maketrans('', '', string.punctuation)
lmtzr = WordNetLemmatizer()

def get_tokens(line):
    '''
    Function to parse text features
    '''
    tokens = word_tokenize(line)
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    # remove punctuations from each word
    stripped = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    # filter out stopwords
    words = [w for w in words if not w in stop_words]
    # lemmatizing the words, see https://en.wikipedia.org/wiki/Lemmatisation
    '''lemmatise or stem???'''
    words = [lmtzr.lemmatize(w) for w in words]
    # remove single letters
    words = [word for word in words if not len(word)==1]
    return (words)

In [ ]:
import pyspark.sql.functions as F
from pyspark.ml import Pipeline, Transformer
from pyspark.ml.feature import Bucketizer
from pyspark.sql import DataFrame
from typing import Iterable
import pandas as pd

## custom transformer to spread sparse vectors into individual columns
class VectorMLliber(Transformer):
    """
    A custom Transformer which converts a column of pyspark.ml vectors to multiple pyspark.mllib vectors.
    """

    def __init__(self, inputCol=None):
        super(VectorMLliber, self).__init__()

    def _transform(self, df: DataFrame) -> DataFrame:
        
        def f(v):
            return Vectors.sparse(v.size, v.indices, v.values)
        
        df = df.rdd.map(lambda r: as_mllib_vector(r[0]))
        return df

In [ ]:
# ???????????
VectorMLliber_body = VectorMLliber(inputCol='body_features')
VectorMLliber_title = VectorMLliber(inputCol='titl_features')

In [ ]:
'''def as_mllib_vector(v):
    return Vectors.sparse(v.size, v.indices, v.values)

features = {}
feature_vec_list = {}
for i in data_array:
    features[i] = word_feat_list[i].select("features")
    feature_vec_list[i] = features[i].rdd.map(lambda r: as_mllib_vector(r[0]))
    feature_vec_list[i].cache()
'''

In [ ]:
(trained_pipeline
 .transform(datasets['english'])
 .select(
    indep_text_variables + ["prediction"]
 )
 .write
 .parquet("linreg_prediction.parquet")
)

In [ ]:
linreg_predictions = spark.read.parquet("linreg_prediction.parquet")

In [ ]:
linreg_predictions.toPandas().head()

In [ ]:
linreg_predictions.select("prediction").describe().toPandas()

# Save pipelines

In [ ]:
from joblib import dump, load
dump(estimator_pipeline, 'pipeline.joblib') 

reloaded = load("pipeline.joblib")

#Now we can predict directly!

reloaded.predict(X)[:10]

In [ ]:
## save models DOESN'T WORK BECAUSE: 'NLTKWordPunctTokenizer' object has no attribute '_to_java'
for i in data_array:
    param_dict[i].save(f'{i}-pipeline') 

# Convert notebook to python file

In [ ]:
!jupyter nbconvert --to script 0-master-notebook-pipelines.ipynb